In [4]:
import sys
sys.path.insert(1, '../..')

import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv("../../data/processed/twitter15-tf_dataset.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,tvt2,tvt2_1,tvt2_2,tvt2_3
0,551099691702956032,rip elly may clampett: so sad to learn #beverl...,True,training,validation,training,training
1,514106273852174337,just in: missing afghan soldiers found trying ...,True,training,training,training,testting
2,495366618818830336,#riphulkhogan my heart is ripping like your sh...,False,training,testting,validation,training
3,532206910796468224,a chick-fil-a manager allegedly banned this hi...,False,validation,training,training,training
4,560187970389819392,islamic tribunal using sharia law in texas has...,False,validation,training,validation,validation


In [6]:
# labels_str = ['rumour', 'non-rumour']
label_type = "label"
labels_str = data[label_type].unique().tolist()
labels_str

[True, False]

In [7]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d[label_type])
    labels.append(lab)
labels[:10]

[0, 0, 1, 1, 1, 1, 1, 1, 0, 1]

In [8]:
import string
import nltk
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(reduce_len=True)


def text2unigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    unigrams = texts
    
    return unigrams


def text2bigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    bigrams = nltk.bigrams(texts)
    bigrams = map(' '.join, bigrams)
    bigrams = [bgr for bgr in bigrams]
    
    return bigrams


def text2trigrams(text):
    texts = tokenizer.tokenize(text.encode('ascii', 'ignore').decode('utf8'))
    texts = [t for t in texts if t not in string.punctuation]
    texts = [t for t in texts if t not in ['URL', '‘', '’']]
    
    trigrams = nltk.trigrams(texts)
    trigrams = map(' '.join, trigrams)
    trigrams = [bgr for bgr in trigrams]
    
    return trigrams


def custom_vectors_generation(texts, vector_terms):
    vectors = []
    for text in texts:
        bigrams = text2bigrams(text)
        trigrams = text2trigrams(text)

        init_vec = [0.0 for _ in range(len(vector_terms) + 1)]
        for bgr in bigrams:
            if bgr in vector_terms:
                idx = vector_terms.index(bgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        for tgr in trigrams:
            if tgr in vector_terms:
                idx = vector_terms.index(tgr)
                init_vec[idx] = 1.0
            else:
                init_vec[-1] = 1.0
        vectors.append(init_vec)
    
    return vectors

In [9]:
texts = data['tweet_text'].tolist()
# vectors = bigrams_vectors_generation(texts)

In [10]:
import nltk
from nltk.collocations import *

top_n = 2000
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_words([])
finder3 = TrigramCollocationFinder.from_words([])

# generating bigram and trigram
for text in texts:
    unigrams = text2unigrams(text)
    bigrams = text2bigrams(text)
    trigrams = text2trigrams(text)
    
    for ngrm in unigrams:
        if ngrm not in finder2.word_fd:
            finder2.word_fd[ngrm] = 0
        finder2.word_fd[ngrm] += 1
        finder2.N += 1
        
        if ngrm not in finder3.word_fd:
            finder3.word_fd[ngrm] = 0
        finder3.word_fd[ngrm] += 1
        finder3.N += 1
    
    for ngrm in bigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder2.ngram_fd:
            finder2.ngram_fd[term] = 0
            
        finder2.ngram_fd[term] += 1

    for ngrm in trigrams:
        term = tuple([i for i in ngrm.split()])
        
        if term not in finder3.ngram_fd:
            finder3.ngram_fd[term] = 0
            
        finder3.ngram_fd[term] += 1
        
# only bigrams that appear 3+ times
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(3)

combined = []
for res in finder2.score_ngrams(bigram_measures.pmi):
    combined.append(res)
for res in finder3.score_ngrams(trigram_measures.pmi):
    combined.append(res)
combined = sorted(combined, key=lambda x: x[1], reverse=True)

In [11]:
print(len(finder2.ngram_fd), len(finder3.ngram_fd))
count = 0
for k, v in combined:
    print(f"{k} - {v}")
    count += 1
    if count >= 10:
        break

614 373
('leaving', 'fifty', 'shades') - 23.541603043652167
('pro', 'era', 't-shirt') - 23.541603043652167
('harry', 'potter', 'book') - 23.12656554437332
('cutting', '5,000', 'jobs') - 22.711528045094475
('posted', 'without', 'comment') - 22.06767185531975
('walmart', 'donates', '10,000') - 21.97456245092827
('5,000', 'jobs', 'hp') - 21.711528045094475
('through', 'ur', 'camera') - 21.711528045094475
('aid', 'worker', 'david') - 21.541603043652167
('scottish', 'railway', 'station') - 21.541603043652167


In [12]:
term_vector_base = [" ".join(c[0]) for c in combined[:top_n]]
vectors = custom_vectors_generation(texts, term_vector_base)

In [13]:
len(vectors[102])

988

In [14]:
min_score = 100
max_score = 0
n_2gram = 0
n_3gram = 0
for k, v in combined[:top_n]:
    min_score = min(min_score, v)
    max_score = max(max_score, v)
    
    if len(k) == 2:
        n_2gram += 1
    
    if len(k) == 3:
        n_3gram += 1
        
print(min_score)
print(max_score)
print(n_2gram)
print(n_3gram)

0.022608672236621885
23.541603043652167
614
373


In [15]:
train_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_vectors = np.array([vectors[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

train_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'training'])
val_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'validation'])
test_labels = np.array([labels[i] for i, d in data.iterrows() if d['tvt2'] == 'testting'])

In [16]:
from sklearn.svm import LinearSVC

from library.classification import SKLearnClassification
from library.evaluation import ConfusionMatrix

svm = LinearSVC()

model = SKLearnClassification(svm, "Support Vector Machine")
print(f"\n--- START ---")
model.train(train_vectors, train_labels, "Phemernr2")

print("Validation Set")
preds = model.predict(val_vectors)

conf_mat = ConfusionMatrix(
    labels=val_labels,
    predictions=preds,
    binary=True
)
conf_mat.evaluate()

print("--- END ---\n")


--- START ---
---> execution time : 0.01 seconds
Validation Set
Binary Class Evaluation

True Positive : 65
False Positive : 25
False Negative : 7
True Negative : 55

Class positive Evaluation
- Precision : 72.222 %
- Recall : 90.278 %
- F1 : 0.80247

Class negative Evaluation
- Precision : 88.71 %
- Recall : 68.75 %
- F1 : 0.77465

Combined Evaluation
- Accuracy : 78.947 %
- Precision : 80.466 %
- Recall : 79.514 %
- F1 : 0.79987
- Average Confidence : 100.0 %
Model, Combined,,,,positive,,,negative,,,
Anonymous, 78.947, 80.466, 79.514, 0.79987, 72.222, 90.278, 0.80247, 88.71, 68.75, 0.77465, 
--- END ---



In [17]:
from scipy.sparse import csr_matrix

coefs = model.model.coef_
if type(coefs) == csr_matrix:
    coefs.toarray().tolist()[0]
else:
    coefs.tolist()
coefs_and_features = list(zip(coefs[0], term_vector_base))

# Most predictive overall
# coefs_and_features = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)
coefs_and_features = sorted(coefs_and_features, key=lambda x: abs(x[0]), reverse=True)
print(f"Total tokens : {len(coefs_and_features)}")

Total tokens : 987


In [18]:
coefs.shape

(1, 988)

In [19]:
coefs_and_features

[(-1.357885188436539, 'michael brown'),
 (-1.24099701242841, 'kkk wednesday'),
 (-1.2037988936098432, 'saudi cleric'),
 (-1.1765316301149902, "paul walker's"),
 (-1.1235456612607435, 'new audio'),
 (-1.1235456352725293, 'parliament hill'),
 (-1.0532260573332761, 'in a'),
 (-1.028808536369875, 'fanged deer'),
 (-1.0196045935258162, 'paul walker'),
 (-0.9496223670111199, 'beheading of'),
 (-0.9440889058060735, 'in south'),
 (-0.9373452936798821, 'in ottawa'),
 (-0.9362881327728291, 'will see'),
 (-0.9362880828732928, 'predicted the'),
 (-0.9362880680778196, 'identified as'),
 (-0.9362880539295072, 'death in'),
 (-0.9362880425119494, 'final score'),
 (-0.9154126974868285, 'we are'),
 (0.8691476148581201, 'florida woman'),
 (-0.8350861834227725, 'colleen hufford'),
 (-0.8209294431703934, 'not be'),
 (0.8147224188268876, 'this is'),
 (-0.8126115488115818, 'goliath birdeater'),
 (-0.797181397213951, 'for a'),
 (-0.7639686376801005, 'beheading suspect'),
 (-0.6952063410255184, 'in lockdown'),

In [20]:
coefs_and_features[:10]

[(-1.357885188436539, 'michael brown'),
 (-1.24099701242841, 'kkk wednesday'),
 (-1.2037988936098432, 'saudi cleric'),
 (-1.1765316301149902, "paul walker's"),
 (-1.1235456612607435, 'new audio'),
 (-1.1235456352725293, 'parliament hill'),
 (-1.0532260573332761, 'in a'),
 (-1.028808536369875, 'fanged deer'),
 (-1.0196045935258162, 'paul walker'),
 (-0.9496223670111199, 'beheading of')]

In [21]:
coefs_and_features[-10:]

[(0.0, 'the beverly'),
 (0.0, "he's a"),
 (0.0, 'is now'),
 (0.0, 'soldier has'),
 (0.0, 'a hoax'),
 (0.0, 'in parliament'),
 (0.0, 'that you'),
 (0.0, 'killed in'),
 (0.0, 'is this'),
 (0.0, 'as the')]

In [22]:
coefs_and_features[-10:]

[(0.0, 'the beverly'),
 (0.0, "he's a"),
 (0.0, 'is now'),
 (0.0, 'soldier has'),
 (0.0, 'a hoax'),
 (0.0, 'in parliament'),
 (0.0, 'that you'),
 (0.0, 'killed in'),
 (0.0, 'is this'),
 (0.0, 'as the')]

In [24]:
n_best = 750
best_tokens = []

# for cf in coefs_and_features[-n_best:]:
#     best_tokens.append(cf[1])

for cf in coefs_and_features[:n_best]:
    best_tokens.append(cf[1])
    
print(best_tokens[:10])
    
with open("../../data/processed/twitter15-tf_best_terms.txt", "w") as f:
    for token in best_tokens:
        f.write(token + "\n")

['michael brown', 'kkk wednesday', 'saudi cleric', "paul walker's", 'new audio', 'parliament hill', 'in a', 'fanged deer', 'paul walker', 'beheading of']
